## Imports

In [1]:
# Read stocks
import yfinance as yf
# Dataframe
import pandas as pd
# To access NaN
import numpy as np

## Constants

In [2]:
SECTOR = 'Technology'
INDUSTRY = 'Semiconductors'
# Stock symbols
SYMBOLS = 'NVDA,TSM,AVGO,AMD,INTC,QCOM,TXN,ADI,MU,ARM,MRVL,MPWR,MCHP,ON,STM'.split(',')
# Criteria + Score (computed)
CRITERIA = 'CR1,CR2,CR3,CR4,CR5,CR6,CR7,CR8,CR9,Score'.split(',')

# Human readable column names mapping for raw data
COLUMN_MAPPING_RAW={'Symbol': 'Symbol', 'Name': 'Name', 'CR1': 'Net Income', 'CR2': 'Return on Assets',
                'CR3': 'Op. Cash Flow', 'CR4': 'Quality of Earn', 'CR5': 'Long Term Debt',
                'CR6': 'Current Ratio', 'CR7': 'New Shares', 'CR8': 'Gross Margin', 'CR9': 'Asset TR'}

# Human readable column names mapping for score data - append Score dictionary item
COLUMN_MAPPING_SCORE = COLUMN_MAPPING_RAW | {'Score':'Score'}

## Utility functions

In [3]:
def net_income(ticker):
    df = ticker.income_stmt
    return df.loc['Net Income'].iloc[0]

def roa(ticker):
    df = ticker.balance_sheet
    avg_assets = (df.loc['Total Assets'].iloc[0] + df.loc['Total Assets'].iloc[1])/2
    return round(net_income(ticker)/avg_assets, 2)

def ocf(ticker):
    df = ticker.cash_flow
    if 'Operating Cash Flow' in df.index:
        return df.loc['Operating Cash Flow'].iloc[0]
    else:
        # Calculate Operating Cash Flow using Free Cash Flow and Captial Expenditure
        # Take the absolute value for Captial Expenditure as yf returns as a negative number
        return df.loc['Free Cash Flow'].iloc[0] + abs(df.loc['Capital Expenditure']).iloc[0]

def ltdebt(ticker):
    df = ticker.balance_sheet
    return (df.loc['Long Term Debt'].iloc[1] - df.loc['Long Term Debt'].iloc[0])

def current_ratio(ticker):
    df = ticker.balance_sheet
    current_ratio_current = df.loc['Total Assets'].iloc[0]/df.loc['Total Liabilities Net Minority Interest'].iloc[0]
    current_ratio_prev = df.loc['Total Assets'].iloc[1]/df.loc['Total Liabilities Net Minority Interest'].iloc[1]
    return round((current_ratio_current - current_ratio_prev), 2)

def new_shares(ticker):
    df = ticker.balance_sheet
    return (df.loc['Common Stock'].iloc[1] - df.loc['Common Stock'].iloc[0])

def gross_margin(ticker):
    df = ticker.income_stmt
    gross_margin_current = df.loc['Gross Profit'].iloc[0]/ df.loc['Total Revenue'].iloc[0]
    gross_margin_prev = df.loc['Gross Profit'].iloc[1]/ df.loc['Total Revenue'].iloc[1]
    return (gross_margin_current - gross_margin_prev)

def asset_turnover_ratio(ticker):
    df_bs = ticker.balance_sheet
    y0, y1, y2 = df_bs.loc['Total Assets'].iloc[0], df_bs.loc['Total Assets'].iloc[1], df_bs.loc['Total Assets'].iloc[2]
    avg_asset_y0 = (y0 + y1)/2
    avg_asset_y1 = (y1 + y2)/2
    
    df_is = ticker.income_stmt
    tot_rvn_y0 = df_is.loc['Total Revenue'].iloc[0]/avg_asset_y0
    tot_rvn_y1 = df_is.loc['Total Revenue'].iloc[1]/avg_asset_y1

    return round((tot_rvn_y0 - tot_rvn_y1), 2)

## Maps Criteria to Function

In [4]:
# Criteria name -> method
criteria_dict = {
    'CR1':net_income,
    'CR2':roa,
    'CR3':ocf,
    'CR5':ltdebt,
    'CR6':current_ratio,
    'CR7':new_shares,
    'CR8':gross_margin,
    'CR9':asset_turnover_ratio
}

## Calculate Piotroski Score

In [5]:
def calculate_piotroski_score():
    # Dictionary to collect 9 criteria
    ps_criteria = {
        'Symbol':[],'Name':[],'CR1':[],'CR2':[],'CR3':[],'CR4':[],'CR5':[],
        'CR6':[],'CR7':[],'CR8':[],'CR9':[]
    }
    
    # Dictionary to collect raw data
    ps_criteria_data = {
        'Symbol':[],'Name':[],'CR1':[],'CR2':[],'CR3':[],'CR4':[],'CR5':[],
        'CR6':[],'CR7':[],'CR8':[],'CR9':[]
    }

    # Loop through symbol list
    for symbol in SYMBOLS:
        ticker = yf.Ticker(symbol)

        # Set symbol and name
        ps_criteria['Symbol'].append(ticker.info['symbol'])
        ps_criteria['Name'].append(ticker.info['longName'])
        
        ps_criteria_data['Symbol'].append(ticker.info['symbol'])
        ps_criteria_data['Name'].append(ticker.info['longName'])

        # Set criteria
        for key, value in criteria_dict.items():
            try:
                # Uses the command pattern to call the appropriate method
                result = value(ticker)
                ps_criteria_data[key].append(result)
                # Special adjustment for CR7 - if there re no new shares, i.e, difference between current and previous is 0 then add 1 as well
                if key == 'CR7':
                    ps_criteria[key].append(1 if result >= 0 else 0)
                else:
                    # Process with other CRs
                    ps_criteria[key].append(1 if result > 0 else 0)
            except (KeyError, IndexError) as err:
                # Error encountered, due to missing data"
                print(err)
                ps_criteria[key].append(0)
                ps_criteria_data[key].append(np.nan)

        # CR4 - handle it differently as it doesn't invoke a method
        # CR4 - Cash flow from operations being greater than net income (quality of earnings)
        if ps_criteria_data['CR3'][-1] > ps_criteria_data['CR1'][-1]:
            ps_criteria['CR4'].append(1)
            ps_criteria_data['CR4'].append(1)
        else:
            # Set criteria and raw data to false (0)
            ps_criteria['CR4'].append(0)
            ps_criteria_data['CR4'].append(0)
    return ps_criteria, ps_criteria_data

## Data Frames for Score and Raw data

In [6]:
ps_criteria, ps_criteria_data = calculate_piotroski_score()
ps_criteria_df = pd.DataFrame(ps_criteria)
# Add ranking scores to get the total score
ps_criteria_df['Score'] = ps_criteria_df[CRITERIA[:-1]].sum(axis=1)
ps_criteria_df

'Long Term Debt'
single positional indexer is out-of-bounds
'Long Term Debt'


,Symbol,Name,CR1,CR2,CR3,CR4,CR5,CR6,CR7,CR8,CR9,Score
0,NVDA,NVIDIA Corporation,1,1,1,1,1,0,1,0,0,6
1,TSM,Taiwan Semiconductor Manufacturing Company Lim...,1,1,1,1,0,1,1,1,1,8
2,AVGO,Broadcom Inc.,1,1,1,1,1,1,1,1,1,9
3,AMD,"Advanced Micro Devices, Inc.",1,1,1,1,1,1,0,1,0,7
4,INTC,Intel Corporation,1,1,1,1,0,1,0,0,0,5
5,QCOM,QUALCOMM Incorporated,1,1,1,1,0,1,0,0,0,5
6,TXN,Texas Instruments Incorporated,1,1,1,0,0,0,1,0,0,4
7,ADI,"Analog Devices, Inc.",1,1,1,1,1,1,1,1,1,9
8,MU,"Micron Technology, Inc.",0,0,1,1,0,0,0,0,0,2
9,ARM,Arm Holdings plc,1,1,1,1,0,1,1,1,0,7


In [7]:
ps_criteria_data_df = pd.DataFrame(ps_criteria_data)
ps_criteria_data_df

,Symbol,Name,CR1,CR2,CR3,CR4,CR5,CR6,CR7,CR8,CR9
0,NVDA,NVIDIA Corporation,4.368000e+09,0.10,5.641000e+09,1,1.243000e+09,-0.36,1.000000e+06,-0.080001,-0.11
1,TSM,Taiwan Semiconductor Manufacturing Company Lim...,9.929234e+11,0.23,1.610599e+12,1,-2.257167e+11,0.06,0.000000e+00,0.079320,0.03
2,AVGO,Broadcom Inc.,1.408200e+10,0.19,1.808500e+10,1,1.436000e+09,0.04,0.000000e+00,0.023847,0.04
3,AMD,"Advanced Micro Devices, Inc.",8.540000e+08,0.01,1.667000e+09,1,7.500000e+08,0.39,-1.000000e+06,0.011939,-0.26
4,INTC,Intel Corporation,1.689000e+09,0.01,1.147100e+10,1,-9.294000e+09,0.04,-5.069000e+09,-0.025714,-0.07
5,QCOM,QUALCOMM Incorporated,7.232000e+09,0.14,1.129900e+10,1,-9.470000e+08,0.15,-2.950000e+08,-0.021414,-0.26
6,TXN,Texas Instruments Incorporated,6.510000e+09,0.22,6.420000e+09,0,-2.389000e+09,-0.06,0.000000e+00,-0.058613,-0.18
7,ADI,"Analog Devices, Inc.",3.314579e+09,0.07,4.817634e+09,1,6.461680e+08,0.05,2.168000e+06,0.013159,0.01
8,MU,"Micron Technology, Inc.",-5.833000e+09,-0.09,1.559000e+09,1,-5.923000e+09,-0.86,-1.000000e+06,-0.542970,-0.25
9,ARM,Arm Holdings plc,5.240000e+08,0.08,7.390000e+08,1,NaN,0.24,0.000000e+00,0.008898,NaN


## Utility method to convert a number to human readable format
#### https://stackoverflow.com/questions/579310/formatting-long-numbers-as-strings

In [8]:
def human_format(num):
    if np.isnan(num):
        return 'nan'
    # Check for zero or else it will return an empty string
    elif num == 0:
        return '0'
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0.'), ['', 'K', 'M', 'B', 'T'][magnitude])

## Add Styles to Score DF

In [9]:
def make_pretty_score(styler):
    # Columns with 0 decimal points
    zero_formats = {}
    for idx in range(1,10):
        zero_formats[f'CR{idx}']='{:.0f}'        
    styler.format(zero_formats)
    
    # Hide index
    styler.hide(axis='index')

    # Left text alignment for Symbol and Name columns
    styler.set_properties(subset=['Symbol', 'Name'], **{'text-align': 'left'})
    # Center alignment for the rest
    styler.set_properties(subset=CRITERIA, **{'text-align': 'center'})

    # Set background gradients
    styler.background_gradient(subset=['Score'], cmap='PiYG')
    
    # Borders
    styler.set_properties(**{'border': '1px solid grey'})
    return styler

def column_formatter_score(name):
    return COLUMN_MAPPING_SCORE[name]

## Apply styles to Score DF

In [10]:
# Add table caption and styles to DF
ps_criteria_df.style.pipe(make_pretty_score).set_caption(f'Piotroski Score {SECTOR} - {INDUSTRY}').set_table_styles(
    [{'selector': 'th.col_heading', 'props': [('text-align', 'center'),
                                              ('color', 'bisque'), ('font-size', '10pt')]},
     {'selector': 'td', 'props': [('font-size', '10pt')]},
     {'selector': 'caption', 'props': [('text-align', 'center'), ('color', 'goldenrod'),
                                       ('font-size', '14pt'), ('font-weight', 'bold')]}]).format_index(column_formatter_score, axis=1) 

Symbol,Name,Net Income,Return on Assets,Op. Cash Flow,Quality of Earn,Long Term Debt,Current Ratio,New Shares,Gross Margin,Asset TR,Score
NVDA,NVIDIA Corporation,1,1,1,1,1,0,1,0,0,6
TSM,Taiwan Semiconductor Manufacturing Company Limited,1,1,1,1,0,1,1,1,1,8
AVGO,Broadcom Inc.,1,1,1,1,1,1,1,1,1,9
AMD,"Advanced Micro Devices, Inc.",1,1,1,1,1,1,0,1,0,7
INTC,Intel Corporation,1,1,1,1,0,1,0,0,0,5
QCOM,QUALCOMM Incorporated,1,1,1,1,0,1,0,0,0,5
TXN,Texas Instruments Incorporated,1,1,1,0,0,0,1,0,0,4
ADI,"Analog Devices, Inc.",1,1,1,1,1,1,1,1,1,9
MU,"Micron Technology, Inc.",0,0,1,1,0,0,0,0,0,2
ARM,Arm Holdings plc,1,1,1,1,0,1,1,1,0,7


## Add Styles to Raw DF

In [11]:
def make_pretty_raw(styler):
    # Columns with 2 decimal points
    styler.format({'CR2':'{:.2f}','CR6':'{:.2f}','CR8':'{:.2f}','CR9':'{:.2f}'})
    
    # Apply human_fomrat for cols
    styler.format(human_format, subset=['CR1','CR3','CR5','CR7'])
    
    # Hide index
    styler.hide(axis='index')

    # Left text alignment for some columns
    styler.set_properties(subset=['Symbol', 'Name'], **{'text-align': 'left'})

    # Borders
    styler.set_properties(**{'border': '1px solid grey'})
    return styler

def column_formatter_raw(name):
    return COLUMN_MAPPING_RAW[name]

## Apply styles to Raw DF

In [12]:
# Add table caption and styles to DF
ps_criteria_data_df.style.pipe(make_pretty_raw).set_caption(f'Data for Piotroski Score {SECTOR} - {INDUSTRY}').set_table_styles(
    [{'selector': 'th.col_heading', 'props': [('text-align', 'center'),
                                              ('color', 'bisque'), ('font-size', '10pt')]},
     {'selector': 'td', 'props': [('font-size', '10pt')]},
     {'selector': 'caption', 'props': [('text-align', 'center'), ('color', 'goldenrod'),
                                       ('font-size', '14pt'), ('font-weight', 'bold')]}]).format_index(column_formatter_raw, axis=1) 

Symbol,Name,Net Income,Return on Assets,Op. Cash Flow,Quality of Earn,Long Term Debt,Current Ratio,New Shares,Gross Margin,Asset TR
NVDA,NVIDIA Corporation,4.37B,0.10,5.64B,1,1.24B,-0.36,1M,-0.08,-0.11
TSM,Taiwan Semiconductor Manufacturing Company Limited,993B,0.23,1.61T,1,-226B,0.06,0,0.08,0.03
AVGO,Broadcom Inc.,14.1B,0.19,18.1B,1,1.44B,0.04,0,0.02,0.04
AMD,"Advanced Micro Devices, Inc.",854M,0.01,1.67B,1,75M,0.39,-1M,0.01,-0.26
INTC,Intel Corporation,1.69B,0.01,11.5B,1,-9.29B,0.04,-5.07B,-0.03,-0.07
QCOM,QUALCOMM Incorporated,7.23B,0.14,11.3B,1,-947M,0.15,-295M,-0.02,-0.26
TXN,Texas Instruments Incorporated,6.51B,0.22,6.42B,0,-2.39B,-0.06,0,-0.06,-0.18
ADI,"Analog Devices, Inc.",3.31B,0.07,4.82B,1,646M,0.05,2.17M,0.01,0.01
MU,"Micron Technology, Inc.",-5.83B,-0.09,1.56B,1,-5.92B,-0.86,-1M,-0.54,-0.25
ARM,Arm Holdings plc,524M,0.08,739M,1,nan,0.24,0,0.01,nan
